In [ ]:
import pandas as pd

In [ ]:
# CSV 파일 불러오기
file_path = 'bunjang_보헴서_여성의류_최종.csv'
df = pd.read_csv(file_path)

In [ ]:
# 1. 컬럼 이름 변경 
rename_dict = {
    'pid': 'id',
    '상품 이름': 'name',
    '가격': 'price',
    '설명': 'description',
    '상품상태': 'item_condition',
    '판매상태': 'sale_status',
    '찜 수': 'likes',
    '조회수': 'views',
    '배송비': 'shipping_fee',
    '사진': 'image_url',
    '카테고리': 'category',
    '직거래지역': 'location',
    '상품태그': 'tags',
    '검수가능': 'inspectable',
    '링크': 'link',
    '업데이트일': 'update_time'
}
df.rename(columns=rename_dict, inplace=True)

In [ ]:
# 2. 숫자형 컬럼 변환 
for col in ['price', 'likes', 'views', 'shipping_fee']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
# 3. 상품 이름 조건 제거 (포함된 문자열 제거) : 
# 구매 희망 게시글(예: "삽니다")은 판매 데이터가 아니므로 제거해야 분석 품질을 높일 수 있음.

remove_keywords = ['구해요', '삽니다', '구합니다', '구매', '사요']
pattern = '|'.join(remove_keywords)
mask = df['name'].str.contains(pattern, case=False, na=False)
df = df[~mask]

In [ ]:
# 4. location: 시군-구까지만 :"서울 강남구 삼성동" -> "서울 강남구" 형태로 축소
df['location'] = df['location'].str.extract(r'([^ ]+ [^ ]+)')

In [ ]:
# 5. 가격 조건 : 너무 낮은 가격(비정상 거래) 및 1원 단위의 판매글은 상품 구매글이 많았음(예: 1004원, 9999원 등)

df = df[df['price'] % 10 == 0]
df = df[df['price'] >= 1000]

In [ ]:
# 6. desc_len 열 추가 : 상품 설명의 길이는 판매 여부와 관련이 있을 수 있어 유용한 파생 변수로 활용하기 위해 컬럼 추가
df['desc_len'] = df['description'].astype(str).apply(len)

In [ ]:
# 7. item_condition ordinal encoding : 번개장터 판매글 등록시 상품 상태를 선택하게 되어 있는데 이를 숫자형 데이터로 처리함.
condition_map = {
    'NEW': 0,
    'LIKE_NEW': 1,
    'LIGHTLY_USED': 2,
    'HEAVILY_USED': 3,
    'USED': 4,
    'DAMAGED': 5
}
df['item_condition'] = df['item_condition'].map(condition_map)

In [ ]:
# 8. 판매상태 Binary 값 설정 : 판매중 = 0 / 예약중, 판매완료 = 1로 변환
# 번개장터의 수수로 정책으로 인해(안전결제) 이미 판매가 완료 되었음에도 예약중으로 걸어두고 판매완료로 바꾸지 않는 상황 반영

def map_sale_status(status):
    if status == 'SELLING':
        return 0
    else:
        return 1

df['sale_status_binary'] = df['sale_status'].apply(map_sale_status)

In [ ]:
# 9. image_count 열 추가 
df['image_count'] = df['image_url'].apply(lambda x: len(str(x).split(';')) if pd.notna(x) else 0)

In [ ]:
# 10. 카테고리 파생 컬럼 추가 : 기존의 긴 카테고리를 주요 분류로 나눠 유의미한 분석이 가능하도록 변경
# middle_category, last_category 생성, 기존 category 컬럼 제거
def split_category(cat):
    parts = str(cat).split('>')
    parts = [p.strip() for p in parts]
    if len(parts) >= 3:
        return parts[1], parts[-1]  # 두 번째와 마지막
    elif len(parts) >= 2:
        return parts[1], parts[-1]
    else:
        return '기타', '기타'

df[['middle_category', 'last_category']] = df['category'].apply(lambda x: pd.Series(split_category(x)))
df.drop(columns=['category'], inplace=True)

In [ ]:
# 11. 상품태그에서 # 제거
def clean_tags(tag_str):
    if pd.isna(tag_str):
        return tag_str
    return tag_str.replace('#', '').strip()

df['tags'] = df['tags'].apply(clean_tags)

In [ ]:
# 인덱스 재정렬
df.reset_index(drop=True, inplace=True)